# Import Libraries

In [3]:
import pandas as pd
import regex as re
import twitter
import nltk
from nltk.tokenize import RegexpTokenizer 

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)

#put in all 4 keys and token via twitter --> you will need to put in your own
consumer_key='mz0i3X2LVYuLGTDTrFBjNKTmP'
consumer_secret='ocmdo8sVOAriRaL4MVPRVa5wzGrkSr4BOnCyAdYawGlXRje4VN'
access_token_key='581353008-EzGKHiZbG2EuXb8lzLSWBnD1q6woN9uCluHv02qb'
access_token_secret='ycpvlXOXa97Lqu0n9wIQSRmkD768cINOMmrrHCmDVAEbm'

api = twitter.Api(consumer_key, consumer_secret, access_token_key, access_token_secret)

In [12]:
def twitter_in(event=None, keywords=None):
        #create query search string
        query = str(event)
        query = (''.join(str(word)+'&' for word in keywords)+query)
        
        #scrape twitter w/query string, returns json
        tweets = api.GetSearch(raw_query="q="+query+"=typd&lang=en", return_json=True)
        
        #make in df
        df = pd.DataFrame(tweets['statuses'])
        
        #pull out the urls
        #creates blank list
        urls = []
        
        #appends urls to blank list
        for index, value in df.entities.iteritems():
            urls.append(re.findall(r"'url': '(\S+)',", str(value)))
            
        #makes list into a new df column
        df['url'] = urls    
        
        #tokenize the tweets
        tokenize_tweets = RegexpTokenizer(r'\w+')
        
        #create a list of tokens
        texts = [tokenize_tweets.tokenize(value) for index, value in df.text.iteritems()]
        
        
        #nake new column for tokens
        df['token_text'] = texts
        
        #append the keywords to the closed words list
        
        #
        closed_words = ['closed', 'flooded', 'inaccessible', 'road', 'stuck', 'flood', 'close', 'drive', 'lane', 'street']
        count_list = []
        for index, word_list in df.token_text.iteritems():
            count = 0
            
            for word in word_list:
                if word.lower() in closed_words:
                    count += 1
                if word.lower() == 'rt':
                    count -= 1
            count_list.append(count) #could add different amounts for certain words
        df['num_closed_words'] = count_list
        
        if df[df.num_closed_words > 1].shape[0] == 0:
            print('no relevant tweets')
        else: 
            return df[df.num_closed_words > 1]

        #return df.num_closed_words.value_counts()

        #return (type(df.entities[1]))


In [13]:
event = 'weather'
keywords = ['4th', 'u', 'road', 'blocked', 'tree', 'down']
twitter_in(event, keywords)

no relevant tweets
